# Demo notebook for radiomics analysis of 18-455E from Conrad XNAT

In [1]:
import os, pydicom, shutil, getpass

from pyxnat import Interface

from oct2py import octave

In [2]:
%load_ext oct2py.ipython

In [3]:
# Local path to CERR Octave code
cerrPath = '/cluster/home/locastre/CERR_octave/20230113/CERR'

# XNAT host address
#xhost = 'http://xnat.mskcc.org'
xhost = 'https://conrad.mskcc.org'

### Connect to XNAT

In [4]:
xusr = 'locastre'
xpwd = getpass.getpass()

 ········


In [7]:
xnat = Interface(xhost, user=xusr, password=xpwd)

In [8]:
proj = '18-455E'
xproj = xnat.select.project(proj)

subj0 = 'RIA_18-455E_000_000001'

xsubj = xproj.subject(subj0)
exp = xsubj.experiments('*').get()[0]
print(exp)

xexp = xsubj.experiment(exp)

slabel = xexp.attrs.get('label')

MSKCC02_E32444


### Define function to pull RTSTRUCT and associated scan from a RIA exam on XNAT

In [9]:
def pull_XNAT_struct_and_DICOM(xexp, workingdir):
    exp = xexp.attrs.get('label')
    xscanlist = xexp.scans('*')
    dcmdir = os.path.join(workingdir,exp)
    os.makedirs(dcmdir,exist_ok = True)
    roidcmfile = ''
    for xroi in xexp.assessors('*'):
        for xres in xroi.resources('*'):
            roifile = xres.files().get()[0]
            if '.dcm' in roifile:
                roidcmfile = os.path.join(dcmdir,roifile)
                xres.file(roifile).get(roidcmfile)
                rtstruct = pydicom.dcmread(roidcmfile)
                seriesUID = rtstruct[0x30060010][0][0x30060012][0][0x30060014][0][0x0020000e].value
        for xscan in xscanlist:
            flist = xscan.resource('DICOM').files().get()
            if flist != []:
                f0 = flist[0]
                xscan.resource('DICOM').file(f0).get(f0)
                fdcm = pydicom.dcmread(f0)
                if str(fdcm.SeriesInstanceUID) == seriesUID:
                    slabel = xscan.attrs.get('ID')
                    print('RTSTRUCT matches ' + slabel)
                    seriesdir = os.path.join(dcmdir,slabel)
                    os.makedirs(seriesdir,exist_ok=True)
                    xscan.resource('DICOM').get(seriesdir,extract = True)
                    destroifile = os.path.join(seriesdir,'DICOM',roifile)
                    if not os.path.exists(destroifile):
                        shutil.move(roidcmfile,os.path.join(seriesdir,'DICOM'))
                    else:
                        os.remove(roidcmfile)
                    break
                os.remove(f0)
    dcmdir = os.path.join(seriesdir,'DICOM')
    return dcmdir, xscan

#### Set download location for DICOM files

In [10]:
downloaddir = '/lab/deasylab1/Eve/rectal_xnat'
os.makedirs(downloaddir,exist_ok = True)

In [11]:
dcmdir, xscan = pull_XNAT_struct_and_DICOM(xexp,downloaddir)

RTSTRUCT matches 7


In [12]:
print('DICOM image and RTSTRUCT files downloaded to ' + dcmdir)

DICOM image and RTSTRUCT files downloaded to /lab/deasylab1/Eve/rectal_xnat/524882/7/DICOM


In [13]:
featureDir = '/lab/deasylab1/Eve/rectal_xnat/524882/7/radfeatures'
os.makedirs(featureDir,exist_ok=True)

In [14]:
%octave_push cerrPath slabel subj0 dcmdir featureDir

### Run CERR feature calculation in Octave as illustrated in https://github.com/stratis-forge/radiomics-workflows/blob/main/radiomic_and_dosimetric_feature_extraction.ipynb

In [15]:
%%octave

pkg load image
pkg load io
pkg load statistics

setenv('JAVA_HOME','/content/jdk1.8.0_211')

addpath(cerrPath);
addToPath2(cerrPath);

disp(dcmdir)
init_ML_DICOM;
planC = importDICOM(dcmdir);

indexS = planC{end};
numel(planC{indexS.structures})

#save_planC(planC,[],'passed',fullfile(dcmdir,[slabel '_' subj0 '.mat']));
#disp(fullfile(dcmdir,[slabel '_' subj0 '.mat']))
#exist(fullfile(dcmdir,[slabel '_' subj0 '.mat']))

success = 1

/lab/deasylab1/Eve/rectal_xnat/524882/7/DICOM

Elapsed time is 0.0242479 seconds.

Elapsed time is 0.00670409 seconds.

ans = 1

...

 Reading segmentLabel ...

 Reading CERROptions ...

 Reading indexS ...

Creating raster-scan representation of unnamed contour roi contour.

ans = 1

In [16]:
%%octave

sampleParam = fullfile(cerrPath,'Jupyter_Notebooks/demoParams.json');
paramS = getRadiomicsParamTemplate(sampleParam);

In [17]:
%%octave
indexS = planC{end};
strC = {planC{indexS.structures}.structureName};
#strName = paramS.structuresC{1};
#structNum = getMatchingIndex(strName,strC,'exact');
structNum = 1;
strName = planC{indexS.structures}(structNum).structureName;
disp(strName)
scanNum = getStructureAssociatedScan(structNum,planC);
disp(scanNum)
doseNum = 1;
binWidth = 0.01;
disp(strName);
structFieldName = ['struct_',repSpaceHyp(strName)];
disp(structFieldName);

Unnamed contour ROI

1

Unnamed contour ROI

struct_Unnamed_contour_ROI

In [18]:
%%octave

radiomicsFeatS = calcGlobalRadiomicsFeatures(scanNum, structNum, paramS, planC);

writeFeaturesToCSV(radiomicsFeatS,[featureDir '/ria1_radiomics_features.csv'],{'RIA1'});

Elapsed time is 0.405679 seconds.

Elapsed time is 0.00500512 seconds.

Elapsed time is 0.00869298 seconds.

### Upload radiomics features CSV to XNAT

In [19]:
xscan = xnat.select.project(proj).subject(subj0).experiment(exp).scan('7')
xscan.exists()

True

In [20]:
scan = xscan.attrs.get('ID')

### Conrad XNAT resource creation

In [21]:
import requests

session = requests.Session()
session.auth = (xusr, xpwd)
auth = session.post(xhost)

#PUT - /data/experiments/{experiment-id}/scans/{scan-id}/resources/{resource-label}

In [22]:
print(exp)
print(subj0)
print(proj)
print(scan)

MSKCC02_E32444
RIA_18-455E_000_000001
18-455E
7


In [23]:
res = 'CERR'
xres = xscan.resource(res)
print(xres.exists())

True


In [ ]:
if not xres.exists():
    req_URL = xhost + '/data/experiments/' + exp + '/scans/' + scan + '/resources/' + res
    print(req_URL)
    r = session.put(req_URL)
    r.content

In [26]:
xfile = xres.file('CERR_radiomics_features.csv')
if not xfile.exists():
    xfile.insert(os.path.join(featureDir,'ria1_radiomics_features.csv'),'CERR_radiomics_features.csv')